Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [13]:
import os
import sys
nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.squad import load_pandas_df
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.models.bert.question_answering import BERTQAExtractor

In [2]:
SQUAD_VERSION = "v1.1" 
CACHE_DIR = "./temp"

LANGUAGE = Language.ENGLISH
DO_LOWER_CASE = True

DOC_TEXT_COL = "doc_text"
QUESTION_TEXT_COL = "question_text"
ANSWER_START_COL = "answer_start"
ANSWER_TEXT_COL = "answer_text"
QA_ID_COL = "qa_id"
IS_IMPOSSIBLE_COL = "is_impossible"

In [3]:
train_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="train")
dev_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="dev")

In [4]:
train_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,5733be284776f4190066117e,False
1,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,908,1987,5733bf84d058e614000b61c1,False
2,The university is the major seat of the Congre...,Which prize did Frederick Buechner create?,675,Buechner Prize for Preaching,5733bed24776f4190066118c,False
3,The College of Engineering was established in ...,The College of Science began to offer civil en...,155,the 1870s,5733a6424776f41900660f52,False
4,All of Notre Dame's undergraduate students are...,Which organization declared the First Year of ...,647,U.S. News & World Report,5733a70c4776f41900660f65,False


In [5]:
dev_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,Super Bowl 50 was an American football game to...,What 2015 NFL team one the AFC playoff?,None,None,56d9895ddc89441400fdb510,False
1,The Panthers finished the regular season with ...,What year did the Carolina Panthers form?,None,None,56d98a59dc89441400fdb52e,False
2,The Broncos took an early lead in Super Bowl 5...,How many tackles did Von Miller accomlish by h...,None,None,56d98b33dc89441400fdb53e,False
3,"CBS broadcast Super Bowl 50 in the U.S., and c...",What performer lead the Super Bowl XLVIII half...,None,None,56d98c53dc89441400fdb548,False
4,"In early 2012, NFL Commissioner Roger Goodell ...",What year did Roger Goodell announce that Supe...,None,None,56d98d0adc89441400fdb54f,False


In [6]:
tokenizer = Tokenizer(language=LANGUAGE, to_lower=DO_LOWER_CASE,cache_dir=CACHE_DIR)

In [20]:
train_features, qa_examples = tokenizer.tokenize_qa(
    doc_text=train_df[DOC_TEXT_COL], 
    question_text=train_df[QUESTION_TEXT_COL], 
    answer_start=train_df[ANSWER_START_COL], 
    answer_text=train_df[ANSWER_TEXT_COL],
    qa_id=train_df[QA_ID_COL],
    is_impossible=train_df[IS_IMPOSSIBLE_COL],
    is_training=True)

In [21]:
sample_feature = train_features[0]
for f in type(sample_feature)._fields:
    print(f)
    print(getattr(sample_feature, f))
    print()

unique_id
1000000000

example_index
0

tokens
['[CLS]', 'what', 'sits', 'on', 'top', 'of', 'the', 'main', 'building', 'at', 'notre', 'dame', '?', '[SEP]', 'architectural', '##ly', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'", 's', 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'up', '##rai', '##sed', 'with', 'the', 'legend', '"', 've', '##ni', '##te', 'ad', 'me', 'om', '##nes', '"', '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'gr', '##otto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'gr', '##otto', 'at', 'lou', '##rdes', ',', 'france', 'where', 'the', 'vi

In [30]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

# This script reuses some code from
# https://github.com/huggingface/pytorch-transformers/blob/067923d3267325f525f4e46f357360c191ba562e/examples/run_squad.py
# https://github.com/huggingface/pytorch-transformers/blob/067923d3267325f525f4e46f357360c191ba562e/examples/utils_squad.py
# https://github.com/huggingface/pytorch-transformers/blob/067923d3267325f525f4e46f357360c191ba562e/examples/utils_squad_evaluate.py


import os
import logging
from tqdm import tqdm, trange

import torch
from torch.utils.data import (
    DataLoader,
    RandomSampler,
    SequentialSampler,
    TensorDataset,
)

from pytorch_transformers import AdamW, WarmupLinearSchedule
from pytorch_transformers.modeling_bert import BertConfig, BertForQuestionAnswering

from utils_nlp.models.bert.common import Language
from utils_nlp.common.pytorch_utils import get_device, move_to_device


logger = logging.getLogger(__name__)

class BERTQAExtractor:
    def __init__(self, language=Language.ENGLISH, cache_dir=".", fine_tuned_model=None):
                
        self.language = language
        self.cache_dir = cache_dir


        if fine_tuned_model:
            self.model = BertForQuestionAnswering.from_pretrained(fine_tuned_model)
        else:
            config = BertConfig.from_pretrained(language.value)
            self.model = BertForQuestionAnswering.from_pretrained(
                language.value, config=config
            )

    def fit(
        self, 
        features,
        num_gpus=None,
        num_epochs=1,
        batch_size=32,
        lr=2e-5,
        warmup_proportion=None,
        max_grad_norm=1.0,
        model_output_dir=None
        ):
        device = get_device("cpu" if num_gpus == 0 or not torch.cuda.is_available() else "gpu")
        self.model = move_to_device(self.model, device, num_gpus)

        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

        all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
        all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
        train_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                      all_start_positions, all_end_positions)

        train_sampler = RandomSampler(train_dataset)
        train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

        t_total = len(train_dataloader) * num_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)

        if warmup_proportion:
            warmup_steps = t_total * warmup_proportion
        else:
            warmup_steps = 0

        scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)

        global_step = 0
        tr_loss, logging_loss = 0.0, 0.0
        self.model.zero_grad()
        train_iterator = trange(int(num_epochs), desc="Epoch")
        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(device) for t in batch)
                inputs = {'input_ids':       batch[0],
                          'attention_mask':  batch[1], 
                          'token_type_ids':  batch[2],  
                          'start_positions': batch[3], 
                          'end_positions':   batch[4]}

                outputs = self.model(**inputs)
                loss = outputs[0]  # model outputs are always tuple in pytorch-transformers

                loss = loss.mean() # mean() to average on multi-gpu parallel (not distributed) training


                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)

                tr_loss += loss.item()

                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                self.model.zero_grad()
                global_step += 1

                if global_step % 50 == 0:
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/50, global_step)
                    logging_loss = tr_loss

            epoch_iterator.close()
        train_iterator.close()

        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

        print ("Average training loss: {}".format(tr_loss / global_step))

        if model_output_dir:
            if not os.path.exists(model_output_dir):
                os.makedirs(model_output_dir)

            logger.info("Saving model checkpoint to %s", model_output_dir)
            # Save a trained model, configuration and tokenizer using `save_pretrained()`.
            # They can then be reloaded using `from_pretrained()`
            model_to_save = self.model.module if hasattr(self.model, 'module') else model  # Take care of distributed/parallel training
            model_to_save.save_pretrained(model_output_dir)


    def predict(
        self,
        features,
        num_gpus=None,
        batch_size=32,
        probabilities=False):
        
        device = get_device("cpu" if num_gpus == 0 or not torch.cuda.is_available() else "gpu")
        self.model = move_to_device(self.model, device, num_gpus)

        # score
        self.model.eval()

        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        test_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                     all_example_index)

        test_sampler = SequentialSampler(test_dataset)
        test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

        all_results = []
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            batch = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'token_type_ids': batch[2]
                        }
                example_indices = batch[3]

                outputs = self.model(**inputs)

            for i, example_index in enumerate(example_indices):
                test_feature = features[example_index.item()]
                unique_id = int(test_feature.unique_id)

                result = RawResult(unique_id    = unique_id,
                                   start_logits = to_list(outputs[0][i]),
                                   end_logits   = to_list(outputs[1][i]))
                all_results.append(result)
qa_extractor = BERTQAExtractor(language=LANGUAGE, cache_dir=CACHE_DIR)

In [23]:
train_features = train_features[0:63]

In [ ]:
qa_extractor.fit(train_features,
                 num_gpus=0,
                 num_epochs=1,
                 batch_size=32,
                 lr=2e-5,
                 warmup_proportion=0.1,
                 max_grad_norm=1.0,
                 model_output_dir=CACHE_DIR)

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]